In [5]:
# !wget https://raw.githubusercontent.com/UMxYTL-AI-Labs/MalayMMLU/refs/heads/main/data/MalayMMLU_0shot.json

In [6]:
import re
import json

In [7]:
with open('MalayMMLU_0shot.json') as fopen:
    d = json.load(fopen)

rejected = ['II', '#', '<', '+', '~', '*']

ABCDE = 'ABCDE'
sound = {
    'A': 'Aee',
    'B': 'Bee',
    'C': 'See',
    'D': 'Dee',
    'E': 'Eee',
}

filtered = []
original = []
for d_ in d:
    if len(re.findall(r'\b\d+\b', d_['prompt'])):
        continue
    
    if any([r in d_['prompt'] for r in rejected]):
        continue
    
    t = d_['prompt'].replace('\n', ' ')
    
    for k, v in sound.items():
        t = t.replace(f'{k}.', f'. {v} . ')
        
    t = re.sub(r'\.{3,}', ', Tempat Kosong , ', t)
    t = re.sub(r'_{3,}', ', Tempat Kosong , ', t)
    
    t = re.sub(r'[ ]+', ' ', t + ' .').strip()
    
    filtered.append({
        'question': d_['prompt'],
        'pronunciation': t,
        'answer': d_['answer']
    })
    original.append(d_)
    
len(filtered)

20437

In [8]:
with open('filter-dialects.json') as fopen:
    speakers = json.load(fopen)

In [9]:
import random

s = random.sample(speakers, k = len(filtered))
for i in range(len(s)):
    t = re.sub(r"<\|.*?\|>", "", s[i]['new_text']).strip()
    s[i]['transcription'] = t
    s[i].pop('new_text', None)
    s[i]['audio'] = s[i].pop('audio_filename')

for i in range(len(filtered)):
    filtered[i]['speaker'] = s[i]
    
with open('malaymmlu.json', 'w') as fopen:
    json.dump(filtered, fopen)
    
with open('filtered-malaymmlu.json', 'w') as fopen:
    json.dump(original, fopen)

In [10]:
filtered[-4]

{'question': 'Analisis IPO adalah untuk menganalpasti data input, proses untuk mengubah nilai data kepada maklumat dan paparan OUTPUT maklumat stelah proses\nA. Benar\nB. Salah',
 'pronunciation': 'Analisis IPO adalah untuk menganalpasti data input, proses untuk mengubah nilai data kepada maklumat dan paparan OUTPUT maklumat stelah proses . Aee . Benar . Bee . Salah .',
 'answer': 'A. Benar',
 'speaker': {'transcription': 'Atau, ia akan mengikut fasa asalnya. Ya. Jom ikuti video ni. Baik, ramai yang mengharapkan, ya,',
  'audio': '/home/husein/ssd3/dialects_processed/MINGGU PENENTUAN BAYARAN STR FASA 3 SEBELUM AIDILADHA. [eWhWsGesfa0]/MINGGU PENENTUAN BAYARAN STR FASA 3 SEBELUM AIDILADHA. [eWhWsGesfa0]_2.mp3'}}

In [1]:
from huggingface_hub import HfApi
api = HfApi()

api.upload_file(
    path_or_fileobj="malaymmlu.json",
    path_in_repo="malaymmlu.json",
    repo_id="mesolitica/Speech-Instructions-MalayMMLU",
    repo_type="dataset",
)

malaymmlu.json:   0%|          | 0.00/14.8M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/Speech-Instructions-MalayMMLU/commit/b2068e4a6f1238e9ada60f5d3e2cd78669e5a76b', commit_message='Upload malaymmlu.json with huggingface_hub', commit_description='', oid='b2068e4a6f1238e9ada60f5d3e2cd78669e5a76b', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/mesolitica/Speech-Instructions-MalayMMLU', endpoint='https://huggingface.co', repo_type='dataset', repo_id='mesolitica/Speech-Instructions-MalayMMLU'), pr_revision=None, pr_num=None)